In [1]:
'''
INSTRUCTIONS:

1. Restart and run all. 

2. Scroll down to see if 'SYSTEM SETUP SUCCESSFUL'

3. Below that, can enter Buy or Sell transactions.

4. Below that, can view Current Portfolio Holdings, Transaction Log and Plots.

'''

In [2]:
import pandas as pd
import numpy as np
import sqlite3
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
from pandas_datareader import data as wb

In [3]:
conn = sqlite3.connect('PortfolioTrackerSQL.db')
c = conn.cursor()

In [4]:
def create_tables():   
        
    # Table of current portfolio holdings
    c.execute('''CREATE TABLE Holdings (
                Ticker text,
                Volume integer,
                AvgBuyPrice real,
                LastPrice real,
                MarketValue real,
                ProfitLoss$ real,
                ProfitLoss text
                )''')
    
    # Track Net Position. Sets initial value to 0.
    c.execute('''CREATE TABLE Position (
              NetPosition real,
              NetPositionPercentage text
              )''')
    
    c.execute("INSERT INTO Position VALUES ('0.0','')")
    
    # Table with log of all transactions
    c.execute('''CREATE TABLE Log (
                DateTime text,
                Type text,
                Ticker text,
                Volume integer,
                Price real,
                CashFlow real
                )''')
    
    conn.commit()

In [5]:
def view_holdings(conn):
    
    # Update data before pulling for visualisation
    update()
    
    # Pull data from Holdings Table
    df = pd.read_sql_query("SELECT * from Holdings", conn)
    df.sort_values('Ticker', inplace=True)
    
    # Plot current holdings    
    Weights = []
    Labels = []
    for i in df.index:
        Weights.append(df['MarketValue'][i])
        Labels.append(df['Ticker'][i])
        
    fig = plt.figure(figsize=(3,3))
    ax = fig.add_axes([0,0,1,1])
    ax.axis('equal')
    ax.pie(Weights, labels=Labels,autopct='%1.2f%%')
    plt.title('Current Portfolio')
    plt.show()
    
    ### PRINT NET POSITION ###
    read_position = pd.read_sql_query("SELECT * from Position", conn)
    Net_Position = round(read_position['NetPosition'][0],2)
    Net_Position_Percentage = read_position['NetPositionPercentage'][0]
    
    # Format output
    
    RED = '\033[31m'
    GREEN =  '\033[32m'
    RESET = '\033[m'
    
    if Net_Position > 0:
        print('Net Position: ' + GREEN + '+' + str(Net_Position) +' (+'+Net_Position_Percentage+')'+ RESET)
    elif Net_Position == 0:
        print('Net Position: ' + str(Net_Position))
    else:
        print('Net Position: ' + RED + str(Net_Position) +' ('+Net_Position_Percentage+')'+ RESET)
    
    ### PRINT MARKET VALUE ###
    market_value = df['MarketValue'].sum()
    print('Market Value: $' + str(round(market_value,2)))
    
    return df

In [6]:
def view_log(conn,rows):
    df = pd.read_sql_query("SELECT * from Log", conn)
    df['DateTime'] = pd.to_datetime(df['DateTime'],format='%d-%m-%Y')
    df.set_index('DateTime',inplace=True)
    df.sort_index(ascending=False,inplace=True)
    return df.head(rows)

In [7]:
def buy_stock(date, buy_ticker, buy_volume, buy_price):
    
    ### UPDATE HOLDINGS TABLE ###
    
    # Check if ticker already exists
    c.execute("SELECT Ticker FROM Holdings")
    all_tickers = c.fetchall()
    existing_tickers = []
    for i in range(len(all_tickers)):
        existing_tickers.append(all_tickers[i][0])
        
    # If it does:
    if buy_ticker in existing_tickers:
        # Calculate new average price
        c.execute("SELECT Volume, AvgBuyPrice FROM Holdings WHERE TICKER = (?)",(buy_ticker,))
        results = c.fetchall()
        old_volume = results[0][0]
        old_price = results[0][1]
        new_average_price = round(((old_volume * old_price + buy_volume * buy_price) / (old_volume + buy_volume)),2)
        
        # Update record VOLUME and PRICE
        new_volume = old_volume + buy_volume
        c.execute("UPDATE Holdings SET Volume=? WHERE Ticker=?", [new_volume, buy_ticker])
        c.execute("UPDATE Holdings SET AvgBuyPrice=? WHERE Ticker=?", [new_average_price, buy_ticker])
         
    # If not, create new record
    elif buy_ticker not in existing_tickers:
        market_value = buy_volume * buy_price
        c.execute("INSERT INTO Holdings VALUES (?,?,?,?,?,?,?)", 
                  (buy_ticker,buy_volume,buy_price,buy_price,market_value,0,0))
       
    ### UPDATE LOG TABLE ###
    
    transaction_cost = -(buy_volume * buy_price)
    
    c.execute("INSERT INTO Log VALUES (?,?,?,?,?,?)", 
              (date, 'Buy',buy_ticker,buy_volume,buy_price,transaction_cost))
    
    update() # updates Market Values, P/L and Net Position
    
    conn.commit()

In [8]:
def sell_stock(date, sell_ticker, sell_volume, sell_price):
    
    # Calculate new volume 
    c.execute("SELECT Volume FROM Holdings WHERE Ticker = (?)",(sell_ticker,))
    try:
        old_volume = c.fetchone()[0]
        new_volume = old_volume - sell_volume
    except TypeError:
        raise SystemExit('ERROR. Unable to sell: Ticker not owned')
    
    if new_volume >= 0:
        ### UPDATE HOLDINGS TABLE

        # Update record, or delete if volume is 0 
        if new_volume == 0:
            c.execute("DELETE from Holdings WHERE Ticker = (?)",(sell_ticker,))
        else:
            c.execute("UPDATE Holdings SET Volume=? WHERE Ticker=?", [new_volume, sell_ticker])
            
            update() # updates Market Values, P/L and Net Position
        
        ### UPDATE LOG TABLE ###
        
        transaction_cost = sell_volume * sell_price
        
        c.execute("INSERT INTO Log VALUES (?,?,?,?,?,?)", 
                  (date, 'Sell',sell_ticker,sell_volume,sell_price,transaction_cost))
          
        conn.commit()
    
    else:
        print('ERROR. Cannot sell more than previously owned.')
    

In [9]:
# Update function called daily, when buying and selling
def update(): 
    
    ### Check what tickers are owned and add to a list ###
    
    df = pd.read_sql_query("SELECT * from Holdings", conn)
    tickers_owned = []
    for i in range(len(df['Ticker'])):
        tickers_owned.append(str(df['Ticker'][i])+'.AX')
    
    ### Download prices ###
    prices = pd.DataFrame()

    # try to download today's prices
    try: 
        date = str(datetime.now().strftime('%Y-%m-%d')) 
        for i in tickers_owned:
            prices[i] = wb.DataReader(i,data_source = 'yahoo',start=date, end=date)['Close']

    # if not available, download last available price
    except KeyError: 
        for i in range(1,100):
            try:
                yesterday = datetime.now() - timedelta(days=i)
                date = str(yesterday.strftime('%Y-%m-%d'))
                for i in tickers_owned:
                    prices[i] = wb.DataReader(i,data_source = 'yahoo',start=date, end=date)['Close']
                break
            
            except KeyError:
                pass
    
    
    ### Create dict with ticker + price ### 
    portfolio = {}
    for i in prices:
        portfolio[i] = round(prices[i][0],2)
    
    
    ### UPDATE ###
  
    for ticker,price in portfolio.items():
        
        ticker = ticker[:-3] #remove .AX from the ticker name
        
        # Update last price
        c.execute("UPDATE Holdings SET LastPrice=? WHERE Ticker=?", [price, ticker])
        
        # Update market value
        c.execute("SELECT Volume FROM Holdings WHERE Ticker = (?)",(ticker,))
        volume = c.fetchone()[0]
        market_value = round(volume * price,2)
        c.execute("UPDATE Holdings SET MarketValue=? WHERE Ticker=?", [market_value, ticker])
        
        # Update P/L
        c.execute("SELECT AvgBuyPrice FROM Holdings WHERE Ticker = (?)",(ticker,))
        average_price = c.fetchone()[0]  
        price_paid = volume * average_price
        PL_dollar = market_value - price_paid
        PL_percent = str(round((market_value/price_paid-1)*100,2))+'%'
        c.execute("UPDATE Holdings SET ProfitLoss$=? WHERE Ticker=?", [PL_dollar, ticker])
        c.execute("UPDATE Holdings SET ProfitLoss=? WHERE Ticker=?", [PL_percent, ticker])
        
    # Update Net Position value (sum of all current profit/loss)
    c.execute("SELECT ProfitLoss$ FROM Holdings")
    results = c.fetchall()
    Net_Position = 0
    for i in range(len(results)):
        Net_Position += results[i][0]
    c.execute("UPDATE Position SET NetPosition=?", [Net_Position])
    
    # Update Net Position Percentage (sum of all current profit/loss as a percentage of initial cost)
    
    try:
        # Find total investment buy cost
        c.execute("SELECT Volume, AvgBuyPrice FROM Holdings")
        results = c.fetchall()  
        total_cost = 0 # Find total investment buy cost
        for i in results:
            item_cost = i[0] * i[1]
            total_cost += item_cost
        # Find current portfolio market value
        c.execute("SELECT MarketValue FROM Holdings")
        results = c.fetchall()
        current_market_value = 0
        for i in results:
            current_market_value += i[0]
        # Calculate net position as a %
        Net_PL = round(((current_market_value / total_cost)-1)*100,2)
        NetPositionPercentage = str(Net_PL)+'%' 
        c.execute("UPDATE Position SET NetPositionPercentage=?", [NetPositionPercentage])
    
    except ZeroDivisionError:
        pass
        
                   
    conn.commit()

In [10]:
# TRACK HISTORICAL PORTFOLIO PERFORMANCE #

def historical_data():
    
    # Retrieve transaction log and set datetime index
    log = pd.read_sql_query("SELECT * from Log", conn)
    log['DateTime'] = pd.to_datetime(log['DateTime'],format='%d-%m-%Y')
    log.set_index('DateTime',inplace=True)
    log

    # Set up dataframe with dates from first transaction made until today
    df = pd.DataFrame()
    df = df.reindex(pd.date_range(start= log.index.min(), 
                                  end= pd.Timestamp.today(), 
                                  freq='1D')) 
    # Make a column for each ticker
    for i in log['Ticker'].unique():
        df[i] = 0

    # Fill in the amounts purchased of each ticker at each date
    transactions = log.reset_index()

    for i in transactions.index:

        date = transactions['DateTime'][i]
        ticker = transactions['Ticker'][i]
        transaction_type = transactions['Type'][i]
        volume = transactions['Volume'][i]

        if transaction_type == 'Buy': 
            df[ticker][date] += volume

        elif transaction_type == 'Sell':            
            df[ticker][date] -= volume


    ''' 
    Populate portfolio holdings for each day.

    Checks each value for the next day. If it does not change (0 ie no change in holdings) it copies previous day's volume.
    If it does change, it adds or subtracts depending on buy or sell transaction.
    '''

    for row in df.index:       
        for column in df.columns:

            # if next day value is not zero, transaction occured therefore amend volume holding
            if df.shift(-1).loc[row][column] != 0:

                try:
                    next_row = row + timedelta(days=1) 
                    df[column][next_row] += df[column][row]

                except KeyError:
                    pass

            # if next day value is zero, no transaction occured therefore volume holing is equal to previous day
            elif df.shift(-1).loc[row][column] == 0:

                try:
                    next_row = row + timedelta(days=1) 
                    df[column][next_row] = df[column][row] 

                except KeyError:
                    pass   

    '''
    Create dataframe and populate with available prices, then reformat to match shape of df with units holding.
    '''
    start = str(log.index.min().strftime('%Y-%m-%d'))
    end = str(pd.Timestamp.today().strftime('%Y-%m-%d'))

    prices = pd.DataFrame()

    for i in range(1,100):
        try:
            yesterday = pd.Timestamp.today() - timedelta(days=i)
            date = str(yesterday.strftime('%Y-%m-%d'))

            for i in df.columns:

                ticker = i + '.AX'

                prices[i] = wb.DataReader(ticker,data_source = 'yahoo',start=start, end=end)['Close']
            break

        except KeyError:
            pass

    prices = prices.reindex(pd.date_range(start= log.index.min(), 
                                          end= pd.Timestamp.today(), 
                                          freq='1D')) 
    prices.ffill(inplace=True)


    # Multiply units held by price for each day then compute total portfolio value for each day
    values = df.mul(prices)
    values['PortfolioValue'] = values[df.columns].sum(axis=1)
    values = values.round(decimals=2)


    ### CREATE NEW DATAFRAME USING ADJ PRICES ###
    '''
    Create dataframe and populate with available prices, then reformat to match shape of df with units holding.
    '''
    start = str(log.index.min().strftime('%Y-%m-%d'))
    end = str(pd.Timestamp.today().strftime('%Y-%m-%d'))

    adj_prices = pd.DataFrame()

    for i in range(1,100):
        try:
            yesterday = pd.Timestamp.today() - timedelta(days=i)
            date = str(yesterday.strftime('%Y-%m-%d'))

            for i in df.columns:

                ticker = i + '.AX'

                adj_prices[i] = wb.DataReader(ticker,data_source = 'yahoo',start=start, end=end)['Adj Close']
            break

        except KeyError:
            pass

    adj_prices = adj_prices.reindex(pd.date_range(start= log.index.min(), 
                                                  end= pd.Timestamp.today(), 
                                                  freq='1D')) 
    adj_prices.ffill(inplace=True)

    # Multiply units held by price for each day then compute total portfolio value for each day
    adj_values = df.mul(adj_prices)
    adj_values['PortfolioValue'] = adj_values[df.columns].sum(axis=1)
    adj_values = adj_values.round(decimals=2)

    '''
    ### FIND AMOUNT PAID FOR EACH DAY THEREFORE TRACK NET POSITION OVER TIME ###
    '''

    # Set up dataframe with dates from first transaction made until today
    costs = pd.DataFrame()
    costs = df.reindex(pd.date_range(start= log.index.min(), 
                                     end= pd.Timestamp.today(), 
                                     freq='1D')) 
    # Make a column for each ticker
    for i in log['Ticker'].unique():
        costs[i] = 0

    # Fill in the amounts purchased of each ticker at each date
    transac = log.reset_index()

    for i in transac.index:

        date = transac['DateTime'][i]
        order = transac['Type'][i]
        ticker = transac['Ticker'][i]
        volume = transac['Volume'][i]
        cashflow = transac['CashFlow'][i]


        if order == 'Buy': 
            costs[ticker][date] = cashflow

        elif order == 'Sell':
            # check all records in log prior to sell date
            previous_records = log.loc[:date]

            # find all records where ticker was bought and store the volumes and prices at each event
            volume_price_pairs = []
            for i in previous_records.index:
                if str(previous_records['Ticker'][i]) == ticker and previous_records['Type'][i] == 'Buy':
                    previous_volume = previous_records['Volume'][i]
                    previous_price = previous_records['Price'][i]
                    volume_price_pairs.append([previous_volume,previous_price])
                else:
                    pass

            # calculate average price
            numerator = 0
            denominator = 0
            for i in volume_price_pairs:
                previous_volume = i[0]
                previous_price = i[1] 
                numerator += previous_volume * previous_price
                denominator += previous_volume
            average_price = numerator / denominator

            costs[ticker][date] = +(volume * average_price)

    '''
    Update total cost for each day based on buy/sell transactions for each day.

    Iterate over all columns except TotalCost.

    '''

    costs['TotalCost'] = 0

    for row in costs.index:

        # if a transaction occured, adjust the total cost
        for column in costs.loc[:, costs.columns != 'TotalCost'].columns:

            if costs.loc[row][column] != 0:
                costs.loc[row]['TotalCost'] += costs.loc[row][column]

        # if no transactions occured, total cost is equal to previous day
        if costs.shift(-1).loc[row]['TotalCost'] == 0:    
            try:
                next_row = row + timedelta(days=1) 
                costs['TotalCost'][next_row] = costs['TotalCost'][row]

            except KeyError:
                pass
               
    '''
    Download ASX data as benchmark
    '''
    start = str(log.index.min().strftime('%Y-%m-%d'))
    end = str(pd.Timestamp.today().strftime('%Y-%m-%d'))
    asx = pd.DataFrame()
    for i in range(1,100):
        try:
            yesterday = pd.Timestamp.today() - timedelta(days=i)
            date = str(yesterday.strftime('%Y-%m-%d'))         
            asx = wb.DataReader('IOZ.AX',data_source = 'yahoo',start=start, end=end)[['Close','Adj Close']]
            break   
        except KeyError:
            pass

    asx = asx.reindex(pd.date_range(start= log.index.min(), 
                                    end= pd.Timestamp.today(), 
                                    freq='1D')) 
    asx.ffill(inplace=True)
    asx['DailyChange'] = (asx['Close'] / asx['Close'].shift(1))-1
    
    
    # Calculate daily P/L and daily change in value
    values['Cost'] = costs['TotalCost']
    values['Position'] = values['PortfolioValue'] + values['Cost']
    values['PosRat'] = values['PortfolioValue'] / -(values['Cost'])
    values['DailyChange'] = (values['PosRat'] / values['PosRat'].shift(1))-1

    adj_values['Cost'] = costs['TotalCost']
    adj_values['Position'] = adj_values['PortfolioValue'] + adj_values['Cost']
    adj_values['PosRat'] = adj_values['PortfolioValue'] / -(adj_values['Cost'])
    adj_values['DailyChange'] = (adj_values['PosRat'] / adj_values['PosRat'].shift(1))-1

    ''' 
    END UP WITH 5 DATAFRAMES:
    0. log --> log of historical transactions
    1. values --> daily portfolio values using Close prices
    2. adj_values --> daily portfolio values using AdjClose prices
    3. costs --> daily portfolio costs (cost paid for portfolio constituents)
    4. asx --> ASX200 data as benchmark for comparison
    '''

    return log, values, adj_values, costs, asx

In [11]:
def plots():
    
    # Gather historical portfolio data used to construct plots 
    data = historical_data()
    log = data[0]
    values = data[1]
    adj_values = data[2]
    costs = data[3]
    asx = data[4]

    # PLOT 1 - Portfolio value vs Benchmark and daily % change
    plt.figure(figsize=(16,10))
    plt.subplot(2,1,1)
    plt.plot(values['PosRat'][2:]/values['PosRat'][2:].iloc[0]*100,c='blue', label = 'Portfolio')
    plt.plot(asx['Close'][2:]/asx['Close'][2:].iloc[0]*100,c='red',ls='--',label = 'ASX200 Benchmark')
    breakeven_x = pd.Series([values.index[2],values.index.max()])
    breakeven_y = pd.Series([100,100])
    plt.plot(breakeven_x, breakeven_y, c='grey',ls='dotted',label='Breakeven') 
    plt.xlabel('Date')
    plt.ylabel('Normalised Value')
    plt.xlim((values.index[2], values.index.max()))
    plt.title('Portfolio Value vs ASX200 Index')
    plt.legend()

    plt.subplot(2,1,2)
    plt.plot(values.index[2:], values['DailyChange'][2:], c='blue', label='Portfolio')
    plt.plot(asx.index[2:], asx['DailyChange'][2:], c='red', ls='--', label = 'ASX200 Benchmark')
    breakeven_x = pd.Series([values.index[2],values.index.max()])
    breakeven_y = pd.Series([0,0])
    plt.plot(breakeven_x, breakeven_y, c='grey',ls='dotted')
    plt.xlabel('Date')
    plt.ylabel('Daily Change %')
    plt.xlim((values.index[2], values.index.max()))
    plt.ylim(( (-1 * float(values['DailyChange'].max())) - 0.01, float(values['DailyChange'].max()) + 0.01))   
    plt.title('Daily % Change in Value')
    plt.legend()

    plt.show()
    
    # PLOT 2 - Portfolio value over time incorporating dividends
    plt.figure(figsize=(16,5))

    plt.plot(values['PosRat'][2:]/values['PosRat'][2:].iloc[0]*100,c='black', label = 'Without Dividends')
    plt.plot(adj_values['PosRat'][2:]/adj_values['PosRat'][2:].iloc[0]*100,ls='dotted', label = 'With Dividends')

    breakeven_x = pd.Series([values.index[2],values.index.max()])
    breakeven_y = pd.Series([100,100])
    plt.plot(breakeven_x, breakeven_y, c='grey',ls='dotted',label='Breakeven') 
    plt.xlabel('Date')
    plt.ylabel('Normalised Value')
    plt.xlim((values.index[2], values.index.max()))
    plt.title('Dividend Portfolio Returns')

    plt.legend()
    plt.show()

In [12]:
# Load up previous session. Run all until here.

raise SystemExit('SYSTEM SETUP SUCCESSFUL')

SystemExit: SYSTEM SETUP SUCCESSFUL

C:\Users\alex-\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3339: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
### BUY STOCK ###

buy_ticker = 'NAB'
buy_volume = 100
buy_price = 15

date = '24-09-2018'  # DAY-MONTH-YEAR FORMAT

#date = str(datetime.now().strftime('%d-%m-%Y')) 
buy_stock(date, buy_ticker, buy_volume, buy_price)

In [ ]:
### SELL STOCK ###

sell_ticker = 'NAB'
sell_volume = 100
sell_price = 10

date = '01-10-2020'  # DAY-MONTH-YEAR FORMAT

#date = str(datetime.now().strftime('%d-%m-%Y')) 
sell_stock(date, sell_ticker,sell_volume,sell_price)

In [ ]:
### VIEW CURRENT PORTFOLIO ###
view_holdings(conn)

In [ ]:
### VIEW TRANSACTIONS LOG ###
view_log(conn,10) # How many rows to show (latest first)

In [ ]:
### VIEW PLOTS OF HISTORICAL PERFORMANCE ###
plots()

In [ ]:
import plotly.graph_objects as go

# Gather historical portfolio data used to construct plots 
data = historical_data()
log = data[0]
values = data[1]
adj_values = data[2]
costs = data[3]
asx = data[4]

df = pd.DataFrame({'Portfolio':values['PosRat'][2:]/values['PosRat'][2:].iloc[0]*100,
                   'Benchmark':asx['Close'][2:]/asx['Close'][2:].iloc[0]*100,
                   'Pfolio Dchange':values['DailyChange'],
                   'Bmark Dchange':asx['DailyChange']})
df['Breakeven']=100
df['Breakeven2']=0
df.loc['2020-09-24', 'Pfolio Dchange'] = 0
df.loc['2020-09-24', 'Bmark Dchange'] = 0

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=df.index[2:],
                         y=df['Portfolio'][2:] ,
                         name='Portfolio',
                         line=(dict(color='blue', width = 2))))

fig.add_trace(go.Scatter(x=df.index[2:],
                         y=df['Benchmark'][2:],
                         name='ASX200 Index' ,
                         line=(dict(color='red' , width = 2, dash='dash'))))

fig.add_trace(go.Scatter(x=df.index[2:],
                         y=df['Breakeven'][2:],
                         name='Breakeven' ,
                         line=(dict(color='grey' , width = 2, dash='dot'))))

fig.update_layout(xaxis_title='Date',
                  yaxis_title='Normalised Value',
                  title={'text':'Portfolio Value vs ASX200 Index',
                         'y':0.87,
                         'x':0.45,
                         'xanchor':'center',
                         'yanchor':'top'})
fig.show()

fig2 = go.Figure()
fig2.add_trace(go.Scatter(x=df.index[2:],
                          y=df['Pfolio Dchange'][2:],
                          name='Portfolio',
                          line=(dict(color='blue', width = 2))))

fig2.add_trace(go.Scatter(x=df.index[2:],
                          y=df['Bmark Dchange'][2:],
                          name='ASX200 Index' ,
                          line=(dict(color='red' , width = 2, dash='dash'))))

fig2.add_trace(go.Scatter(x=df.index[2:],
                         y=df['Breakeven2'][2:] ,
                         name='Breakeven' ,
                         line=(dict(color='grey' , width = 2, dash='dot'))))

fig2.update_layout(xaxis_title='Date',
                  yaxis_title='% Daily Change',
                  title={'text':'Portfolio Value % Daily Change vs ASX200 Index',
                         'y':0.87,
                         'x':0.45,
                         'xanchor':'center',
                         'yanchor':'top'},
                  yaxis=dict(range=
                             [(-1 * float(df['Pfolio Dchange'].max())) - 0.01, 
                              float(df['Pfolio Dchange'].max()) + 0.01]))

fig2.show()

In [ ]:
# COMPUTE DRAWDOWN #
# 1. Compute wealth index (cumulative returns)
# 2. Compute previous peaks (cumulative max)
# 3. Compute drawdown (wealth value as a percentage of the previous peak)

wealth_index = 100 * (1+ df['Pfolio Dchange']['2020-09-24':]).cumprod()
previous_peaks = wealth_index.cummax()
drawdown = (wealth_index - previous_peaks)/previous_peaks

In [ ]:
plt.figure(figsize=(16,10))
plt.subplot(2,1,1)
wealth_index.plot()
plt.title('Portfolio Cumulative Value')
plt.ylabel('Normalised Value')

plt.subplot(2,1,2)
drawdown.plot.area(color='orangered',alpha=0.5)
plt.title('Drawdown')
plt.ylabel('Drawdown')

plt.tight_layout()
plt.show()

In [ ]:
# MAX DRAWDOWN #
max_drawdown = drawdown.min()
max_drawdown_date = drawdown.idxmin()
max_dd_peak = previous_peaks.loc[max_drawdown_date]
length_max_dd = -1
for i in previous_peaks.index:
    if previous_peaks.loc[i] == max_dd_peak:
        length_max_dd += 1

# % DAYS POSITIVE GROWTH # 
pos_count = len(list(filter(lambda x: (x > 0), values['DailyChange'])))
neg_count = len(list(filter(lambda x: (x < 0), values['DailyChange']))) 
pos_ratio = round((pos_count / (pos_count + neg_count)* 100),2) 

# % DAYS OUTPERFORMED BENCHMARK # 
over_count = 0
under_count = 0
for i in values.index:
    if values.loc[i]['DailyChange'] > asx.loc[i]['DailyChange']:
        over_count += 1
    elif values.loc[i]['DailyChange'] < asx.loc[i]['DailyChange']:
        under_count += 1
over_ratio = round((over_count / (over_count + under_count))* 100,2) 

In [ ]:
print('Max drawdown: ' + str(round(max_drawdown*100,2)) + '%')
print('Max drawdown date: ' + str(max_drawdown_date.strftime('%Y-%m-%d')))
print('Max Drawdown Length: ' + str(length_max_dd) + ' days')
print('--------------------------------------------')
print('No. days Positive growth: ' + str(pos_count))
print('No. days Negative growth: ' + str(neg_count))
print('Positive growth ratio: ' + str(pos_ratio) + '%')
print('--------------------------------------------')
print('No. days Overperformed Benchmark: ' + str(over_count))
print('No. days Underperformed Benchmark: ' + str(under_count))
print('Overperformance ratio: ' + str(over_ratio) + '%')

In [ ]:
# Reset DB
c.execute('DROP TABLE Holdings')
c.execute('DROP TABLE Position')
c.execute('DROP TABLE Log')
create_tables()
view_log(conn,100)

In [ ]:
# reset current portfolio #
buy_stock('22-09-2020','NAB',200,16.73)
buy_stock('24-09-2020','AIA',154,6.555)
buy_stock('24-09-2020','SYD',172,5.936)
view_log(conn,100)